In [63]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
import random
from keras import losses, optimizers


In [64]:
# Cargar datos desde CSV
data = pd.read_csv('C:/Users/LEHI/Documents/Practicas profesionales/ABRIL/preprocessed data 18 columnas - etiqueta letras.csv')


In [65]:
df_uniq = data.groupby('clase').head(20)

In [66]:
#df_uniq

In [67]:
"""
def one_shot_data(dataset_path):
    # Cargar el dataset
    dataset = pd.read_csv(dataset_path)
    
    # Inicializar un diccionario para almacenar un solo ejemplo por clase
    one_shot_data = {}
    
    # Obtener las clases únicas del dataset
    classes = dataset['clase'].unique()
    
    # Seleccionar un solo ejemplo por clase
    for cls in classes:
        class_data = dataset.loc[dataset['clase'] == cls].iloc[0].drop('clase')
        one_shot_data[cls] = class_data.values  # Convertir a un array de numpy
    return one_shot_data
"""

"\ndef one_shot_data(dataset_path):\n    # Cargar el dataset\n    dataset = pd.read_csv(dataset_path)\n    \n    # Inicializar un diccionario para almacenar un solo ejemplo por clase\n    one_shot_data = {}\n    \n    # Obtener las clases únicas del dataset\n    classes = dataset['clase'].unique()\n    \n    # Seleccionar un solo ejemplo por clase\n    for cls in classes:\n        class_data = dataset.loc[dataset['clase'] == cls].iloc[0].drop('clase')\n        one_shot_data[cls] = class_data.values  # Convertir a un array de numpy\n    return one_shot_data\n"

In [68]:
# Dividir los datos en características (X) y etiquetas (y)
X = df_uniq.drop('clase', axis=1)
y = df_uniq['clase']

In [69]:
# Convertir etiquetas a números
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [70]:
# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos (opcional pero recomendado)
X_train = (X_train - X_train.mean()) / X_train.std()
X_test = (X_test - X_test.mean()) / X_test.std()



In [71]:
"""
# Preparar pares de datos para one-shot learning
def create_pairs(X, digit_indices):
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(12)]) - 1
    for d in range(12):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[X.iloc[z1].values, X.iloc[z2].values]]
            labels += [1 if d == i else 0 for i in range(12)]
    return np.array(pairs), np.array(labels)
"""

'\n# Preparar pares de datos para one-shot learning\ndef create_pairs(X, digit_indices):\n    pairs = []\n    labels = []\n    n = min([len(digit_indices[d]) for d in range(12)]) - 1\n    for d in range(12):\n        for i in range(n):\n            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]\n            pairs += [[X.iloc[z1].values, X.iloc[z2].values]]\n            labels += [1 if d == i else 0 for i in range(12)]\n    return np.array(pairs), np.array(labels)\n'

In [72]:
# Definir la arquitectura de la red siamesa
input_shape = X_train.shape[1]


In [73]:
# Definir función para crear la rama de la red siamesa
def create_siamese_branch(input_shape):
    input_layer = tf.keras.layers.Input(shape=(input_shape,))
    x = tf.keras.layers.Dense(64, activation='relu')(input_layer)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    return tf.keras.Model(inputs=input_layer, outputs=x)


In [74]:
# Crear la rama izquierda y derecha de la red siamesa
left_input = tf.keras.layers.Input(shape=(input_shape,))
right_input = tf.keras.layers.Input(shape=(input_shape,))

left_branch = create_siamese_branch(input_shape)(left_input)
right_branch = create_siamese_branch(input_shape)(right_input)

In [75]:
# Concatenar las salidas de ambas ramas
merged = tf.keras.layers.Concatenate()([left_branch, right_branch])

In [76]:
# Agregar capas adicionales y la capa de salida
x = tf.keras.layers.Dense(32, activation='relu')(merged)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(12, activation='softmax')(x)  

In [77]:
# Construir el modelo
model = tf.keras.Model(inputs=[left_input, right_input], outputs=output)

In [78]:
# Compilar el modelo
model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])


In [79]:
# Entrenar el modelo
history = model.fit([X_train, X_train], y_train, epochs=50, batch_size=16, validation_split=0.2)


Epoch 1/50
10/10 [==============================] - 1s 29ms/step - loss: 2.4600 - accuracy: 0.1373 - val_loss: 2.4326 - val_accuracy: 0.2051
Epoch 2/50
10/10 [==============================] - 0s 5ms/step - loss: 2.3191 - accuracy: 0.2418 - val_loss: 2.2717 - val_accuracy: 0.3846
Epoch 3/50
10/10 [==============================] - 0s 5ms/step - loss: 2.1486 - accuracy: 0.3464 - val_loss: 2.1265 - val_accuracy: 0.4615
Epoch 4/50
10/10 [==============================] - 0s 5ms/step - loss: 2.0036 - accuracy: 0.4248 - val_loss: 1.9843 - val_accuracy: 0.4872
Epoch 5/50
10/10 [==============================] - 0s 6ms/step - loss: 1.8393 - accuracy: 0.4771 - val_loss: 1.8358 - val_accuracy: 0.5385
Epoch 6/50
10/10 [==============================] - 0s 5ms/step - loss: 1.7118 - accuracy: 0.5686 - val_loss: 1.6996 - val_accuracy: 0.5897
Epoch 7/50
10/10 [==============================] - 0s 5ms/step - loss: 1.6213 - accuracy: 0.5752 - val_loss: 1.5637 - val_accuracy: 0.6154
Epoch 8/50
10/10 [=

In [80]:
# Cargar datos desde CSV
dataEvaluar = pd.read_csv('C:/Users/LEHI/Documents/Practicas profesionales/ABRIL/preprocessed data 18 columnas - etiqueta letras.csv')

# Dividir los datos en características (X) y etiquetas (y)
X = dataEvaluar.drop('clase', axis=1)
y = dataEvaluar['clase']

In [87]:
#dataEvaluar

In [81]:
# Convertir etiquetas a números
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [82]:
# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Normalizar los datos (opcional pero recomendado)
X_train = (X_train - X_train.mean()) / X_train.std()
X_test = (X_test - X_test.mean()) / X_test.std()


In [83]:
# Evaluar el modelo
y_pred = model.predict([X_test, X_test])
y_pred_classes = np.argmax(y_pred, axis=1)

4/4 [==============================] - 0s 1ms/step


In [84]:
# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred_classes)
print("Matriz de Confusión:")
print(conf_matrix)

Matriz de Confusión:
[[ 7  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  6  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 12  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  9  0  0  0  0  0  0  0  0]
 [ 0  0  0  1 13  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  9  0  1  0  0  0  0]
 [ 0  0  0  0  0  0 12  0  0  0  0  0]
 [ 0  0  0  0  0  1  0 14  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 14  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  4  0  0]
 [ 0  2  0  0  0  0  0  0  0  0  9  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  7]]


In [85]:
#Metricas

with np.errstate(divide='ignore', invalid='ignore'):
    # Exactitud (Accuracy)
    accuracy = np.sum(np.diag(conf_matrix)) / np.sum(conf_matrix)
    print("Exactitud (Accuracy):\n", accuracy)

    # Precisión (Precision)
    precision = np.diag(conf_matrix) / np.sum(conf_matrix, axis=0)
    precision[np.isnan(precision)] = 0  # Reemplaza NaN con 0
    print("Precisión (Precision) por clase:\n", precision)

    # Sensibilidad (Recall)
    sensibilidad = np.diag(conf_matrix) / np.sum(conf_matrix, axis=1)
    sensibilidad[np.isnan(sensibilidad)] = 0  # Reemplaza NaN con 0
    print("Sensibilidad (Recall) por clase:\n", sensibilidad)

    # Puntuación F1 (F1 Score)
    f1_score = 2 * (precision * sensibilidad) / (precision + sensibilidad)
    f1_score[np.isnan(f1_score)] = 0  # Reemplaza NaN con 0
    print("Puntuación F1 (F1 Score) por clase:\n", f1_score)

Exactitud (Accuracy):
 0.9354838709677419
Precisión (Precision) por clase:
 [0.875      0.75       1.         0.9        1.         0.9
 0.92307692 0.93333333 1.         0.8        1.         1.        ]
Sensibilidad (Recall) por clase:
 [0.875      1.         0.92307692 1.         0.92857143 0.81818182
 1.         0.93333333 1.         1.         0.81818182 1.        ]
Puntuación F1 (F1 Score) por clase:
 [0.875      0.85714286 0.96       0.94736842 0.96296296 0.85714286
 0.96       0.93333333 1.         0.88888889 0.9        1.        ]
